In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
#loading dataset
DDP = pd.read_csv("DDG2P_6_10_2023.csv")
DDP

In [ ]:
#Replacing na values with "Unkown"
DDP["organ specificity list"] = DDP["organ specificity list"].fillna("Unknown")

In [ ]:
#Checking to see if previous code worked
DDP[DDP["organ specificity list"].str.contains("Unknown", na=False)]

In [ ]:
#Seperating out rows into individual organs

DDP['organ specificity list'] = DDP['organ specificity list'].str.split(';')

# Explode the column to create multiple rows
DDPsep = DDP.explode('organ specificity list')

# Reset the index if needed
DDPsep = DDPsep.reset_index(drop=True)

DDPsep

In [ ]:
#Collapsing sub organs into main organ

replacement_mapping = {
    'Eye: Retina': 'Eye',
    'Eye: Lens': 'Eye',
    'Eye: Structure': 'Eye',
    'Eye: Physiology': 'Eye',
    'Eye: Cornea': 'Eye',
    'Eye: Connective tissue': 'Eye',
    'Eye: Optic nerve': 'Eye',
    'Eye: Vasculature': 'Eye',
    'Eye: Vitreous': 'Eye',
    'Endocrine': 'Endocrine/Metabolic',
    'Peripheral nerves': 'Spinal cord/Peripheral nerves'
}

DDPsep['organ specificity list'] = DDPsep['organ specificity list'].replace(replacement_mapping)

# The 'organ.specificity.list' column now contains the updated values

In [ ]:
DDPLoc = DDPsep[['gene symbol', 'organ specificity list']]
DDPLoc

In [ ]:
#Merging suplicate listings

DDPsep = DDPsep.groupby('gene symbol')['organ specificity list'].apply(lambda x: ','.join(x)).reset_index()
DDPsep = DDPsep.rename(columns={'organ specificity list': 'organ specificity list'})
DDPsep

In [ ]:
# Define the genes and substrings to search for
genes = DDPsep['gene symbol']
substrings = ["Eye", "Brain", "Skeleton", "Endocrine", "Heart", "Face", "Skin", "Kidney", "GI", "Ear",
              "Bone Marrow", "Musculature", "Genitalia", "Peripheral", "Teeth", "Respiratory", "Hair",
              "Multisystem", "Cancer", "Liver", "Lungs", "Unknown"]

# Create a new DataFrame with genes vs. boolean organ columns
organ_columns = {substring: DDPsep['organ specificity list'].str.contains(substring) for substring in substrings}
organ_columns['gene symbol'] = genes

genes_vs_organs = pd.DataFrame(organ_columns)
# Remove row names
genes_vs_organs = genes_vs_organs.reset_index(drop=True)

# Set the "gene.symbol" column as the new row names
genes_vs_organs.set_index('gene symbol', inplace=True)

In [ ]:
genes_vs_organs

In [ ]:
genes_vs_organs.to_csv("geneVSorgans.csv")

In [ ]:
pwd